/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd




In [3]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                


In [4]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip

Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               


In [5]:
physical_devices = tf.config.list_physical_devices('GPU')

2023-02-22 05:28:33.489099: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 05:28:33.490026: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 05:28:33.707082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 05:28:33.707938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 05:28:33.708786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [6]:
data = pd.read_csv('/kaggle/working/train.csv')

In [7]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
data.count(axis=1,level=None,numeric_only=True)

0         6
1         6
2         6
3         6
4         6
         ..
159566    6
159567    6
159568    6
159569    6
159570    6
Length: 159571, dtype: int64

In [9]:
x=data['toxic'].value_counts()[0]
x


144277

In [10]:
y=data['toxic'].value_counts()[1]
y


15294

In [11]:
x = data['comment_text']
x

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

### 

preprosseing


In [12]:
from tensorflow.keras.layers import TextVectorization 

In [13]:
X=data['comment_text']
Y=data[data.columns[2:]]



In [14]:
MAX_FETURES=200000

In [15]:
vecterizor = TextVectorization(max_tokens=MAX_FETURES,output_sequence_length=1800,output_mode='int')

2023-02-22 05:28:35.916484: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 05:28:36.164679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 05:28:36.165520: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 05:28:36.166233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [16]:
type(X)


pandas.core.series.Series

In [17]:
type(X.values)

numpy.ndarray

In [18]:
vecterizor.adapt(X.values)

2023-02-22 05:28:40.885913: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [19]:
vecterizor.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [20]:
vectorized_text=vecterizor(X.values)

2023-02-22 05:29:00.412360: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.


In [21]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [22]:
TextVectorization??


Init signature: TextVectorization(*args, **kwargs)
Source:        
class TextVectorization(base_preprocessing_layer.PreprocessingLayer):
  """Text vectorization layer.

  This layer has basic options for managing text in a Keras model. It
  transforms a batch of strings (one example = one string) into either a list of
  token indices (one example = 1D tensor of integer token indices) or a dense
  representation (one example = 1D tensor of float values representing data
  about the example's tokens).

  If desired, the user can call this layer's `adapt()` method on a dataset.
  When this layer is adapted, it will analyze the dataset, determine the
  frequency of individual string values, and create a 'vocabulary' from them.
  This vocabulary can have unlimited size or be capped, depending on the
  configuration options for this layer; if there are more unique values in the
  input than the maximum vocabulary size, the most frequent terms will be used
  to create the vocabulary.

  The p

yveygywtev

In [23]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [24]:
dataset.as_numpy_iterator().next()
(batch_x,batch_y)=dataset.as_numpy_iterator().next()

2023-02-22 05:29:02.197736: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.
2023-02-22 05:29:04.448443: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.


In [25]:
batch_x.shape

(16, 1800)

In [26]:
batch_y.shape

(16, 6)

competing the pieline by particining the data set into train test and vald


In [27]:
train=dataset.take(int(len(dataset)*7))
vald=dataset.skip(int(len(dataset)*7)).take(int(len(dataset)*2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,Bidirectional,Embedding

In [29]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FETURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [30]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          6400032   
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                16640     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 7

In [36]:
from keras.callbacks import EarlyStopping

In [37]:
es = EarlyStopping(monitor='val_loss',patience = 3, verbose=1,restore_best_weights=True)

In [41]:
history = model.fit(train, epochs=75,validation_data=vald,callbacks=[es])

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt


In [ ]:
plt.figure(figsize=(8,2))
pd.DataFrame(history.history).plot()
plt.show()

predition time

In [ ]:
batch=test.as_numpy_iterator().next() 

In [ ]:
input_text=vecterizor('Fuck you BITCH. your are a Nigga')

In [ ]:
input_text

In [ ]:
np.expand_dims(input_text,0)

In [ ]:
data.colums[2:]

In [ ]:
model.predict(np.expand_dims(input_text,0))

In [ ]:
model.predict(np.expand_dims(input_text,0))

In [ ]:
batch_X,batch_Y=test.as_numpy_iterator().next() 

In [ ]:
(model.predict(batch_X)>0.5).astype(int)

In [ ]:
batch_Y

In [ ]:
res=model.predict(np.expand_dims(input_text,0))

In [ ]:
res

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')